In [151]:
from sklearn import preprocessing
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shutil
import os
import requests
import base64


# Encode text values to dummy variables(i.e. [1,0,0],[0,1,0],[0,0,1] for red,green,blue)
# 더미 변수에 텍스트 값을 넣음.
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])

    
    for x in dummies.columns:
        dummy_name = "{}-{}".format(name, x)
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)


# Encode text values to a single dummy variable.  The new columns (which do not replace the old) will have a 1
# at every location where the original column (name) matches each of the target_values.  One column is added for
# each target value.
# 텍스트 값을 단일 더미 변수로 인코딩. 새 열은 원래 열(이름)이 각 target_values와 일치하는 모든 위치에 1을 갖습니다. 하나의 열이 
# 각 목표값에 대해 추가됩니다.
def encode_text_single_dummy(df, name, target_values):
    for tv in target_values:
        l = list(df[name].astype(str))
        l = [1 if str(x) == str(tv) else 0 for x in l]
        name2 = "{}-{}".format(name, tv)
        df[name2] = l


# Encode text values to indexes(i.e. [1],[2],[3] for red,green,blue).
# 텍스트 벨류를 인덱스에 인코딩한다.
def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_


# Encode a numeric column as zscores
# 숫자 열을 z 점수로서 인코딩한다.
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()
        
    if sd is None:
        sd = df[name].std()
        
    df[name] = (df[name] - mean) / sd



# Convert all missing values in the specified column to the median
# 지정된 열의 누락된 값들을 평균값을 변경한다.
def missing_median(df, name):
    med = df[name].median()
    df[name] = df[name].fillna(med)


# Convert all missing values in the specified column to the default
# 지정된 열의 누락된 값들을 기본값으로 변경한다.
def missing_default(df, name, default_value):
    df[name] = df[name].fillna(default_value)


# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
# Pandas 데이터프레임을 x,y 입력값(TensorFlow가 필요한)으로 변환한다.
def to_xy(df, target):
    result = []

    for x in df.columns:
        if x != target:
            result.append(x)

    # find out the type of the target column.  Is it really this hard? :(
    # 목표 열의 타입을 찾아라.
    target_type = df[target].dtypes
#     print("Atarget_type:")
#     print(target_type)
    target_type = target_type[0]if hasattr(target_type, '__iter__') else target_type
#     print("Btarget_type:")
#     print(target_type)
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    # 분류를 위해 int로 인코드하고 그렇지 않으면 float으로 한다. 32비트 TensorFlow와 같이.
#     print("np.int64:")
#     print(np.int64)
#     print("np.int32:")
#     print(np.int32)

    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
#         print("dummies")
#         print(dummies)
#         print("A")
        print("before")
        print(dummies.as_matrix())
        print("after")
        return df.as_matrix(result).astype(np.float32), dummies.as_matrix().astype(np.float32)
    else:
        # Regression
#         print("B")
        return df.as_matrix(result).astype(np.float32), df.as_matrix([target]).astype(np.float32)

# Nicely formatted time string
# 지정된 숫자 문자열
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)


# Regression chart.
# Regression 차트
def chart_regression(pred,y,sort=True):
    t = pd.DataFrame({'pred' : pred, 'y' : y.flatten()})
    if sort:
        t.sort_values(by=['y'],inplace=True)
    a = plt.plot(t['y'].tolist(),label='expected')
    b = plt.plot(t['pred'].tolist(),label='prediction')
    plt.ylabel('output')
    plt.legend()
    plt.show()

# Remove all rows where the specified column is +/- sd standard deviations
# 지정된 열이 +/- 표준 편차인 모든 행 제거.
def remove_outliers(df, name, sd):
    drop_rows = df.index[(np.abs(df[name] - df[name].mean()) >= (sd * df[name].std()))]
    df.drop(drop_rows, axis=0, inplace=True)


# Encode a column to a range between normalized_low and normalized_high.
# normalized low와 normalized high 사이의 범위로 인코딩한다.
def encode_numeric_range(df, name, normalized_low=-1, normalized_high=1,
                         data_low=None, data_high=None):
    if data_low is None:
        data_low = min(df[name])
        data_high = max(df[name])

    df[name] = ((df[name] - data_low) / (data_high - data_low)) \
               * (normalized_high - normalized_low) + normalized_low
        
# This function submits an assignment.  You can submit an assignment as much as you like, only the final
# submission counts.  The paramaters are as follows:
# data - Pandas dataframe output.
# key - Your student key that was emailed to you.
# no - The assignment class number, should be 1 through 1.
# source_file - The full path to your Python or IPYNB file.  This must have "_class1" as part of its name.  
# .             The number must match your assignment number.  For example "_class2" for class assignment #2.

def submit(data,key,no,source_file=None):
    if source_file is None and '__file__' not in globals(): raise Exception('Must specify a filename when a Jupyter notebook.')
    if source_file is None: source_file = __file__
    suffix = '_class{}'.format(no)
    if suffix not in source_file: raise Exception('{} must be part of the filename.'.format(suffix))
    with open(source_file, "rb") as image_file:
        encoded_python = base64.b64encode(image_file.read()).decode('ascii')
    ext = os.path.splitext(source_file)[-1].lower()
    if ext not in ['.ipynb','.py']: raise Exception("Source file is {} must be .py or .ipynb".format(ext))
    r = requests.post("https://api.heatonresearch.com/assignment-submit",
        headers={'x-api-key':key}, json={'csv':base64.b64encode(data.to_csv(index=False).encode('ascii')).decode("ascii"),
        'assignment': no, 'ext':ext, 'py':encoded_python})
    if r.status_code == 200:
        print("Success: {}".format(r.text))
    else: print("Failure: {}".format(r.text))


In [152]:
import csv 

# df = open('KDDTrain+.csv','r',encoding='utf-8')
df = pd.read_csv('KDDTrain+.csv', header=None)
# 열 43
print("Read {} rows.".format(len(df)))

Read 125973 rows.


In [153]:
df.dropna(inplace=True, axis=1)

In [154]:
df.columns = [
    'duration',
    'protocol_type',
    'service',
    'flag',
    'src_bytes',
    'dst_bytes',
    'land',
    'wrong_fragment',
    'urgent',
    'hot',
    'num_failed_logins',
    'logged_in',
    'num_compromised',
    'root_shell',
    'su_attempted',
    'num_root',
    'num_file_creations',
    'num_shells',
    'num_access_files',
    'num_outbound_cmds',
    'is_host_login',
    'is_guest_login',
    'count',
    'srv_count',
    'serror_rate',
    'srv_serror_rate',
    'rerror_rate',
    'srv_rerror_rate',
    'same_srv_rate',
    'diff_srv_rate',
    'srv_diff_host_rate',
    'dst_host_count',
    'dst_host_srv_count',
    'dst_host_same_srv_rate',
    'dst_host_diff_srv_rate',
    'dst_host_same_src_port_rate',
    'dst_host_srv_diff_host_rate',
    'dst_host_serror_rate',
    'dst_host_srv_serror_rate',
    'dst_host_rerror_rate',
    'dst_host_srv_rerror_rate',
    'outcome',
    'question'
]

In [155]:
# correlationAttributeEval 낮은 순서대로 제거.
del df['question'] 
del df['num_outbound_cmds'] 
del df['srv_count']
del df['is_host_login']
del df['urgent'] 
del df['num_failed_logins']
del df['land']
del df['num_shells']
# del df['num_compromised']
# del df['num_root']
# del df['hot']
# del df['root_shell']
# del df['num_file_creations']
# del df['su_attempted']
# del df['num_access_files']
# del df['is_guest_login']
# del df['duration']
# del df['dst_host_srv_diff_host_rate']
# del df['protocol_type']
# del df['dst_host_same_src_port_rate']
# del df['wrong_fragment']

# del df['dst_bytes']
# del df['src_bytes']







In [156]:
df[0:5]

,duration,protocol_type,service,flag,src_bytes,dst_bytes,wrong_fragment,hot,logged_in,num_compromised,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,outcome
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal
1,0,udp,other,SF,146,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
2,0,tcp,private,S0,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune
3,0,tcp,http,SF,232,8153,0,0,1,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
4,0,tcp,http,SF,199,420,0,0,1,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal


In [157]:
ENCODING = 'utf-8'

def expand_categories(values):
    result = []
    s = values.value_counts()
    # 특별한 값의 수를 포함한 객체
    t = float(len(values))
    for v in s.index:
        result.append("{}:{}%".format(v,round(100*(s[v]/t),2)))
    return "[{}]".format(",".join(result))
        
def analyze(filename):
    print()
    print("Analyzing: {}".format(filename))
    df = pd.read_csv(filename,encoding=ENCODING)
    cols = df.columns.values
    total = float(len(df))

    print("{} rows".format(int(total)))
    for col in cols:
        uniques = df[col].unique()
        # 해당 열의 특정 값을 리스트로 보여준다.
        unique_count = len(uniques)
        if unique_count>100:
            print("** {}:{} ({}%)".format(col,unique_count,int(((unique_count)/total)*100)))
        else:
            print("** {}:{}".format(col,expand_categories(df[col])))
            expand_categories(df[col])
            

In [158]:
# Analyze KDD-99

import tensorflow.contrib.learn as skflow
import pandas as pd
import os
import numpy as np
from sklearn import metrics
from scipy.stats import zscore

path = "./data/"

# 경로를 병합하여 새 경로 생성
filename_read = os.path.join(path,"auto-mpg.csv")
# ./data/auto-mpg.csv 로 경로 설정

In [159]:
# Now encode the feature vector
# feature 벡터로 인코딩한다.

# 숫자 열을 z 점수로서 인코딩한다. -> 숫자
encode_numeric_zscore(df, 'duration')
# 더미 변수에 텍스트 값을 넣음. -> 문자
encode_text_dummy(df, 'protocol_type')
encode_text_dummy(df, 'service')
encode_text_dummy(df, 'flag')
encode_numeric_zscore(df, 'src_bytes')
encode_numeric_zscore(df, 'dst_bytes')
# encode_text_dummy(df, 'land')
encode_numeric_zscore(df, 'wrong_fragment')
# encode_numeric_zscore(df, 'urgent')
encode_numeric_zscore(df, 'hot')
# encode_numeric_zscore(df, 'num_failed_logins')
encode_text_dummy(df, 'logged_in')
encode_numeric_zscore(df, 'num_compromised')
encode_numeric_zscore(df, 'root_shell')
encode_numeric_zscore(df, 'su_attempted')
encode_numeric_zscore(df, 'num_root')
encode_numeric_zscore(df, 'num_file_creations')
# encode_numeric_zscore(df, 'num_shells')
encode_numeric_zscore(df, 'num_access_files')
# encode_numeric_zscore(df, 'num_outbound_cmds')
# encode_text_dummy(df, 'is_host_login')
encode_text_dummy(df, 'is_guest_login')
encode_numeric_zscore(df, 'count')
# encode_numeric_zscore(df, 'srv_count')
encode_numeric_zscore(df, 'serror_rate')
encode_numeric_zscore(df, 'srv_serror_rate')
encode_numeric_zscore(df, 'rerror_rate')
encode_numeric_zscore(df, 'srv_rerror_rate')
encode_numeric_zscore(df, 'same_srv_rate')
encode_numeric_zscore(df, 'diff_srv_rate')
encode_numeric_zscore(df, 'srv_diff_host_rate')
encode_numeric_zscore(df, 'dst_host_count')
encode_numeric_zscore(df, 'dst_host_srv_count')
encode_numeric_zscore(df, 'dst_host_same_srv_rate')
encode_numeric_zscore(df, 'dst_host_diff_srv_rate')
encode_numeric_zscore(df, 'dst_host_same_src_port_rate')
encode_numeric_zscore(df, 'dst_host_srv_diff_host_rate')
encode_numeric_zscore(df, 'dst_host_serror_rate')
encode_numeric_zscore(df, 'dst_host_srv_serror_rate')
encode_numeric_zscore(df, 'dst_host_rerror_rate')
encode_numeric_zscore(df, 'dst_host_srv_rerror_rate')
# 텍스트 value를 index에 인코딩한다.
outcomes = encode_text_index(df, 'outcome')
# print("A")
# print(outcomes)
num_classes = len(outcomes)

# display 5 rows

df.dropna(inplace=True,axis=1)
df[0:5]
# This is the numeric feature vector, as it goes to the neural net
# 숫자 feature 벡터, 신경망이다.

,duration,src_bytes,dst_bytes,wrong_fragment,hot,num_compromised,root_shell,su_attempted,num_root,num_file_creations,...,flag-S0,flag-S1,flag-S2,flag-S3,flag-SF,flag-SH,logged_in-0,logged_in-1,is_guest_login-0,is_guest_login-1
0,-0.110249,-0.007679,-0.004919,-0.089486,-0.095075,-0.011664,-0.036652,-0.024436,-0.012385,-0.02618,...,0,0,0,0,1,0,1,0,1,0
1,-0.110249,-0.007737,-0.004919,-0.089486,-0.095075,-0.011664,-0.036652,-0.024436,-0.012385,-0.02618,...,0,0,0,0,1,0,1,0,1,0
2,-0.110249,-0.007762,-0.004919,-0.089486,-0.095075,-0.011664,-0.036652,-0.024436,-0.012385,-0.02618,...,1,0,0,0,0,0,1,0,1,0
3,-0.110249,-0.007723,-0.002891,-0.089486,-0.095075,-0.011664,-0.036652,-0.024436,-0.012385,-0.02618,...,0,0,0,0,1,0,0,1,1,0
4,-0.110249,-0.007728,-0.004814,-0.089486,-0.095075,-0.011664,-0.036652,-0.024436,-0.012385,-0.02618,...,0,0,0,0,1,0,0,1,1,0


In [160]:
# total_rows = df['duration'].count
# print(total_rows)

In [161]:
import pandas as pd
import io
import requests
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn import metrics
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.callbacks import EarlyStopping

# Break into X (predictors) & y (prediction)
# X( 예측 변수) & y( 예측값)으로 나누기
x, y = to_xy(df,'outcome')

# print("A")
# print(x)
# print(x.shape)

# print("B")
# print(y)
# print(y.shape)

# Create a test/train split.  25% test
# Split into train/test
# test와 train을 나눈다. 
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.25, random_state=42)

print(y_test)


before
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
after


C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:101: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:103: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [162]:
# model = sequential()
# model.add()

# Create neural net
# neural net을 만든다.
# 레이어 구성
# conv 구조와 relu
import time

start_vect=time.time()

model = Sequential()
model.add(Dense(10, input_dim=x.shape[1], kernel_initializer='normal', activation='relu'))
model.add(Dense(50, input_dim=x.shape[1], kernel_initializer='normal', activation='relu'))
model.add(Dense(10, input_dim=x.shape[1], kernel_initializer='normal', activation='relu'))
model.add(Dense(1, kernel_initializer='normal'))
model.add(Dense(y.shape[1], activation='softmax'))
# 학습 프로세스 설정
model.compile(loss='categorical_crossentropy', optimizer='adam')
# 학습 데이터로 학습.
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')
model.fit(x_train,y_train,validation_data=(x_test,y_test),callbacks=[monitor],verbose=2,epochs=100)


Train on 94479 samples, validate on 31494 samples
Epoch 1/100
 - 3s - loss: 0.6465 - val_loss: 0.3501
Epoch 2/100
 - 3s - loss: 0.2923 - val_loss: 0.2641
Epoch 3/100
 - 3s - loss: 0.2380 - val_loss: 0.2364
Epoch 4/100
 - 3s - loss: 0.2181 - val_loss: 0.2236
Epoch 5/100
 - 3s - loss: 0.2033 - val_loss: 0.2172
Epoch 6/100
 - 3s - loss: 0.1951 - val_loss: 0.2025
Epoch 7/100
 - 2s - loss: 0.1880 - val_loss: 0.2046
Epoch 8/100
 - 3s - loss: 0.1818 - val_loss: 0.2001
Epoch 9/100
 - 2s - loss: 0.1789 - val_loss: 0.1929
Epoch 10/100
 - 3s - loss: 0.1728 - val_loss: 0.1889
Epoch 11/100
 - 2s - loss: 0.1669 - val_loss: 0.1856
Epoch 12/100
 - 2s - loss: 0.1654 - val_loss: 0.1882
Epoch 13/100
 - 2s - loss: 0.1612 - val_loss: 0.1854
Epoch 14/100
 - 2s - loss: 0.1591 - val_loss: 0.1784
Epoch 15/100
 - 2s - loss: 0.1552 - val_loss: 0.1817
Epoch 16/100
 - 3s - loss: 0.1537 - val_loss: 0.1805
Epoch 17/100
 - 2s - loss: 0.1526 - val_loss: 0.1712
Epoch 18/100
 - 2s - loss: 0.1508 - val_loss: 0.1787
Epoch

In [163]:
cnt = 0
for y in y_test:
    if y[0] == 1:
        cnt+=1
        
print (y_test)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [164]:
# Measure accuracy
pred = model.predict(x_test)
pred = np.argmax(pred,axis=1)
y_eval = np.argmax(y_test,axis=1)
score = metrics.accuracy_score(y_eval, pred)
print("Validation score: {}".format(score))
print("training Runtime: %0.2f Minutes"%((time.time() - start_vect)/60))

Validation score: 0.9799009335111449
training Runtime: 2.14 Minutes


In [165]:
model.save('keras_NSLKDD.h5')